In [30]:
!pip install -q transformers sentence-transformers faiss-cpu torch cohere

In [31]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import torch
import pandas as pd
import cohere  # Add Cohere for text generation
import os

In [32]:
# Initialize the Cohere client (replace with your API key)
cohere_api_key = "USe_ypur_own_key_to_cross_check_coz_it's_restricted"  # Add your Cohere API key here
co_client = cohere.Client(cohere_api_key)

In [33]:
# Load the dataset (replace with your own dataset or use a sample one)
data = [
    "The company's revenue grew by 15% in the last quarter.",
    "Customer satisfaction scores improved from 85% to 92%.",
    "The new product launch increased market share by 5%.",
    "Employee turnover rate decreased from 12% to 8% annually.",
    "The company expanded operations to three new countries in Europe."
]
df = pd.DataFrame(data, columns=['text'])



In [34]:
# Initialize the embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')



In [35]:
# Create FAISS index
dimension = embed_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)



In [36]:
# Function to embed and add data to FAISS
def embed_and_add(texts):
    embeddings = embed_model.encode(texts)
    index.add(np.array(embeddings).astype('float32'))
    return embeddings



In [37]:
# Embed and add the dataset
embeddings = embed_and_add(df['text'].tolist())

print("Data embedded and added to FAISS index.")



Data embedded and added to FAISS index.


In [38]:
# Save the models and FAISS index
output_dir = 'rag_qa_model'
os.makedirs(output_dir, exist_ok=True)



In [39]:
# Save embedding model
embed_model.save(f'{output_dir}/embed_model')



In [40]:
# Save FAISS index
faiss.write_index(index, f'{output_dir}/faiss_index.bin')



In [41]:
# Save the dataframe
df.to_csv(f'{output_dir}/data.csv', index=False)

print(f"Embedding model and FAISS index saved to {output_dir}")



Embedding model and FAISS index saved to rag_qa_model


In [42]:
def rag_qa(query, top_k=3):
    # Embed the query
    query_embedding = embed_model.encode([query])[0]

    # Retrieve relevant documents from FAISS
    _, I = index.search(np.array([query_embedding]).astype('float32'), top_k)

    # Prepare context
    context = "\n\n".join([df['text'].iloc[i] for i in I[0]])

    # Construct the input prompt
    input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"

    # Use Cohere to generate an answer
    response = co_client.generate(
        model='command-xlarge-nightly',  # Use an accessible model
        prompt=input_text,
        max_tokens=150,
        temperature=0.7,
        k=0,
        stop_sequences=["\n"]
    )

    # Extract the generated answer
    answer = response.generations[0].text.strip()

    return answer


In [43]:
# Example queries to test the model
queries = [
    "How did the company's revenue change recently?",
    "What happened to customer satisfaction?",
    "Did the company expand its operations?",
    "How did the new product launch affect the company?",
    "What changed in terms of employee turnover?"
]



In [44]:
# Run example queries and print results
print("Testing the RAG QA model with example queries:\n")
for query in queries:
    print(f"Query: {query}")
    answer = rag_qa(query)
    print(f"Answer: {answer}\n")

print("RAG QA model test complete.")


Testing the RAG QA model with example queries:

Query: How did the company's revenue change recently?
Answer: The company's revenue increased by 15% in the last quarter, indicating a positive financial growth trend. This growth could be attributed to various factors such as successful sales strategies, increased demand for their products or services, or effective marketing campaigns. The 15% revenue growth is a significant achievement and suggests that the company is expanding and performing well in the market.

Query: What happened to customer satisfaction?
Answer: Customer satisfaction scores improved from 85% to 92%.

Query: Did the company expand its operations?
Answer: Yes, the company expanded its operations to three new countries in Europe.

Query: How did the new product launch affect the company?
Answer: The new product launch had a positive impact on the company, contributing to a 5% increase in market share. This success, along with the expansion into three new European coun